In [1]:
from skimage.feature import hog
from skimage.transform import pyramid_gaussian
from sklearn.metrics import precision_recall_curve
import joblib
from skimage import color
from imutils.object_detection import non_max_suppression
import matplotlib.pyplot as plt
import imutils
import numpy as np
import cv2
import os
import glob

In [2]:
orientations = 9
pixels_per_cell = (16, 16)
cells_per_block = (1, 1)
threshold = .3

#### define silding window

In [3]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y: y + windowSize[1], x: x + windowSize[0]])

In [4]:
model = joblib.load('svm.npy')

In [5]:
scale = 0
detections = []
img= cv2.imread("images/test/0-35.jpg")

#### Apply sliding window

In [6]:
(winW, winH)= (100,100)
windowSize=(winW,winH)
downscale=1.5

In [7]:
for resized in pyramid_gaussian(img, downscale=1.5):
    for (x,y,window) in sliding_window(resized, stepSize=10, windowSize=(winW,winH)):
        if window.shape[0] != winH or window.shape[1] !=winW:
            continue
        a, b, c = window.shape
        if c != 3:
            break
        window = color.rgb2gray(window)
        fds = hog(window, orientations, pixels_per_cell, cells_per_block, block_norm='L2')
        fds = fds.reshape(1, -1)
        pred = model.predict(fds)
        
        if pred == 1:
            if model.decision_function(fds) > 6:
                print("Starfish Detected at: ({}, {})".format(x, y))
                print("Scale ->  {} | Confidence Score {} \n".format(scale,model.decision_function(fds)))
                detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), model.decision_function(fds),
                                   int(windowSize[0]*(downscale**scale)),
                                      int(windowSize[1]*(downscale**scale))))
    scale+=1

Starfish Detected at: (370, 610)
Scale ->  0 | Confidence Score [6.10884876] 



D:\Anaconda\lib\site-packages\skimage\_shared\utils.py:348: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  return func(*args, **kwargs)


#### non-max suppression

In [8]:
clone = resized.copy()
# for (x_tl, y_tl, _, w, h) in detections:
#     cv2.rectangle(img, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 255), thickness = 2)
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections])
sc = [score[0] for (x, y, score, w, h) in detections]
sc = np.array(sc)
pick = non_max_suppression(rects, probs = sc, overlapThresh = 0.65)


In [9]:
for (xA, yA, xB, yB) in pick:
    cv2.rectangle(img, (xA, yA), (xB, yB), (0,255,0), 2)

In [10]:
cv2.imwrite('images/result/result.png',img)

True